In [1]:
import numpy as np
import ipywidgets as wg
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

# Ising model function definitions

In [2]:
def energy_calc(lattice):
    ''' Calculated total energy of the lattice.
    
        Parameters:
            lattice: [N,N] numpy array containing only 1 and -1 values representing the spin configuration.
        
        Returns: dtype float
            Energy of the spin configuration'''
    
    a = lattice[:,1:]*lattice[:,:N-1]
    b = lattice[:N-1,:]*lattice[1:,:]
    # To ensure periodic boundary conditions the last column is multiplied with the first column of the lattice array. 
    # Same is done with the rows.
    c = lattice[0]*lattice[N-1]
    d = lattice[:,0]*lattice[:,N-1]
    # Energy:
    E = -1*(np.sum(a)+np.sum(b)+np.sum(c)+np.sum(d))
    return(E)

In [3]:
def energy_nn(lattice,x,y):
    ''' Calculate nearest neighbour interaction energy of 1 spin in the lattice.
    
        Parameters:
            lattice: dtype numpy array
                [N,N] numpy array containing only 1 and -1 values representing the spin configuration.
            
            x, y: dtype int
                The coordinates of the of the spin of interest
        
        Returns: dtype float
             nearest neighbour interaction energy of the selected spin '''
    
    s = lattice[x,y]
    nn = lattice[(x+1)%N,y] + lattice[(x-1)%N,y] + lattice[x,(y+1)%N] + lattice[x,(y-1)%N]
    E = -1*nn*s
    return(E)

In [4]:
def specific_heat(x):
    ''' Calculates specific heat.
        
        Parameters: 
            x: dtype numpy array
                One dimensional array containing the energy data (or equivalent quantity)
            
        Returns: dtype numpy array
            One dimensional array containing the specific heat data'''
    
    cv = (np.var(x))/(T**2)
    return(cv)

In [5]:
def susceptiblity(mag):
    ''' Calculates specific heat.
        
        Parameters: 
            mag: dtype numpy array
                One dimensional array containing the magnetization data
            
        Returns: dtype numpy array
            One dimensional array containing the susceptibility data'''
    
    susc = (np.var(mag))/T
    return(susc)

In [6]:
def metropolis(lattice,E,M):
    ''' Implement metropolis algorithm for a single spin flip.
    
        Parameters: 
            lattice: dtype numpy array
                [N,N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    # Select random spin (propose trial lattice with probability 1/L^2):
    x = np.random.randint(0,N)
    y = np.random.randint(0,N)
    trial_lattice = np.copy(lattice)
    trial_lattice[x,y] *= -1
    # Calculate energy difference between the old and trial configurations:
    dE = energy_nn(trial_lattice,x,y) - energy_nn(lattice,x,y)
    # Implement second stage of the metropolis algorithm:
    if dE <= 0:
        E = E + dE
        M = M - lattice[x,y] + trial_lattice[x,y] # substract old spin value and add new spin value to the magnetisation
        lattice[x,y] = trial_lattice[x,y]
    else:
        r = np.random.uniform(0,1)
        #p=np.exp(-dE/tau)
        if dE == 4:
            p = exp[1]
        else:
            p = exp[0]
        if r < p:
            E = E + dE
            M = M - lattice[x,y] + trial_lattice[x,y] # substract old spin value and add new spin value to the magnetisation
            lattice[x,y] = trial_lattice[x,y]
    return(lattice,E,M)

In [7]:
def Wolffbacktrack(x,y,cluster,visited):
    ''' Part of the Wolff algorithm, it checks whether a spin is already in the cluster'''
    if cluster[x,y]:
        return False
    if visited[x,y]:
        return True#False
    return True

def grow_cluster(lattice,x,y,ClusterSpin,E,M, cluster,visited):
    """ Starting from a randomly chosen spin, creates a cluster with the Wolff Algorithm.
        
        Parameters:
            lattice: dtype numpy array
                The [N,N] array representing the spin configuration
            
            x, and y: dtype int
                The coordinates of the spin that is added to the cluster
                
            ClusterSpin: dtype int
                The spin value of the cluster
            
            E and M: dtype int or float
                The energy and magnetization of the lattice
            
            cluster: dtype boolean numpy array
                [N,N] boolean numpy array indicating which spins are in the cluster
                
            visited: dtype boolean numpy array
                [N,N] boolean numpy array indicating which spins have already been trialed by the algorithm
                
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization"""
    
    old_E = energy_nn(lattice,x,y)
    lattice[x,y] *=-1
    dE = energy_nn(lattice,x,y) - old_E
    E = E + dE
    M = M - lattice[x,y]*-1 +lattice[x,y]
    #cluster.append([x,y])
    cluster[x,y] = True
    if Wolffbacktrack((x+1)%N,y, cluster, visited):
        E, M = try_add(lattice,(x+1)%N,y,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack((x-1)%N,y, cluster,visited):
        E, M = try_add(lattice,(x-1)%N,y,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack(x,(y+1)%N, cluster,visited):
        E, M = try_add(lattice,x,(y+1)%N,ClusterSpin,E,M, cluster,visited)
    if Wolffbacktrack(x,(y-1)%N, cluster,visited):
        E, M = try_add(lattice,x,(y-1)%N,ClusterSpin,E,M, cluster,visited)
    return(lattice,E,M)

def try_add(lattice,x,y,ClusterSpin,E,M, cluster, visited):
    """Checks of neighbor belongs to cluster, if so then add neighbor to cluster with a certain probability"""
    
    s_help = lattice[x,y]
    #global tau
    if ((s_help<0) != (ClusterSpin<0)):
        r = np.random.uniform(0,1)
        if r < (1-A_wolff):
            lattice, E, M = grow_cluster(lattice,x,y,ClusterSpin,E,M, cluster,visited)
        else:
            visited[x,y] = True
    return(E,M)

def Wolff(lattice,E,M):
    ''' Implements the wolff algorithm for a single timestep.
        
        Parameters: 
            lattice: dtype numpy array
                [N,N] array representing the lattice
            E: dtype float or int
                Energy of the lattice
            M: dtype float or int
                Magnetization of the lattice
        Returns: dtype tuple
            Tuple containing the updated lattice and the corresponding energy and magnetization'''
    
    x = np.random.randint(0,N)
    y = np.random.randint(0,N)
    cluster = np.full((N,N), False, dtype=bool)
    visited = np.full((N,N), False, dtype=bool)
    clusterspin = lattice[x,y]*-1
    lattice, E, M = grow_cluster(lattice,x,y,clusterspin, E, M, cluster,visited)
    return(lattice,E,M)

# 1D correspondence function definitions

In [8]:
def oneD_create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the 1D lattice
    
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map: dtype numpy array
                [N] boolean array containing True if bond is present or False if not. For example spin lattice [1, -1, 1, 1]
    """
    lat = np.array(lat)
    a = lat[1:]-lat[0:len(lat)-1]; b = lat[0]-lat[len(lat)-1]
    c = np.append(a,b)
    f_map = (c == 0)
    return(f_map)

In [9]:
def oneD_backtrack(x,vis,fm,cl):
    """ Part of the 1D cluster decomposition. It recursively checks whether neighbouring spin should be added to the cluster
    """
    if not vis[x]:
        vis[x] = True
        cl.append(x)
        if fm[x]:
            vis, cl = oneD_backtrack((x+1)%len(fm),vis,fm,cl)
        if fm[(x-1)%len(fm)]:
            vis, cl = oneD_backtrack((x-1)%len(fm),vis,fm,cl)
    return(vis,cl)

In [10]:
def oneD_cluster_decomp(lat):
    """ Decomposes the 1D spin lattice into clusters, i.e. creates list of cluster arrays. A cluster array contains the indices 
        of the spins in a cluster
        
        Input:
            lat: dtype numpy array
                [N] array containing only 1 and -1 values representing the spin configuration
                
        Returns:
            cls: dtype list
                list of cluster arrays. Each cluster array contains the indices of the spins in that cluster
    """
    visited = np.full((len(lat)), False)
    frozen_map = oneD_create_frozen_map(lat)
    #zeros = np.where(np.array(lat)==0)[0]
    #tried = dict.fromkeys(zeros,False)
    cls = []
    for i in range(len(lat)):
        clu = []
        visited, clu = oneD_backtrack(i,visited,frozen_map,clu)
        if len(clu) > 0:
            cls.append(clu)
    return(cls)

# 2D correspondence function definitions

In [11]:
def create_frozen_map(lat):
    """ Creates a boolean map of (freezes) all bonds (nn interactions) in the lattice
    
        Input:
            lat: dtype numpy array
                [N,N] array containing only 1 and -1 values representing the spin configuration
            
        Returns:
            f_map_x: dtype numpy array
                [N,N] boolean array containing True if nn bond is present in the x-direction (horizontal), or False if not.
                
            f_map_y: dtype numpy array
                [N,N] boolean array containing True if nn bond is present in the y-direction (vertical), or False if not.
    """
    lat = np.array(lat)
    a = lat[:,1:]-lat[:,:-1]; b = lat[:,0]-lat[:,-1]
    b = np.reshape(b,(len(lat),1))
    c = np.append(a,b,axis=1)
    f_map_y = (c == 0)
    # same for y
    a = lat[1:]-lat[:-1]; b = lat[0]-lat[-1]
    b = np.reshape(b,(1,len(lat)))
    c = np.append(a,b,axis=0)
    f_map_x = (c == 0)
    return(f_map_x, f_map_y)

In [12]:
def backtrack(x,y,vis,fmx,fmy,cl):
    """ Part of the cluster decomposition. It recursively checks whether neighbouring spin should be added to the cluster
    """
    if not vis[x,y]:
        vis[x,y] = True
        cl.append([x,y])
        if fmx[x,y]:
            vis, cl = backtrack((x+1)%len(fmx),y,vis,fmx,fmy,cl)
        if fmx[(x-1)%len(fmx),y]:
            vis, cl = backtrack((x-1)%len(fmx),y,vis,fmx,fmy,cl)
        if fmy[x,y]:
            vis, cl = backtrack(x,(y+1)%len(fmy),vis,fmx,fmy,cl)
        if fmy[x,(y-1)%len(fmy)]:
            vis, cl = backtrack(x,(y-1)%len(fmy),vis,fmx,fmy,cl)
    return(vis,cl)

In [13]:
def cluster_decomp(lat):
    """ Decomposes the spin lattice into clusters, i.e. creates list of cluster arrays. A cluster array contains the indices of 
        the spins in a cluster
        
        Input:
            lat: dtype numpy array
                [N,N] array containing only 1 and -1 values representing the spin configuration
                
        Returns:
            cls: dtype list
                list of cluster arrays. Each cluster array contains the indices of the spins in that cluster
    """
    visited = np.full((len(lat),len(lat)), False)
    frozen_map_x, frozen_map_y = create_frozen_map(lat)
    #zeros = np.where(np.array(lat)==0)[0]
    #tried = dict.fromkeys(zeros,False)
    cls = []
    for i in range(len(lat)):
        for j in range(len(lat)):
            clu = []
            visited, clu = backtrack(i,j,visited,frozen_map_x,frozen_map_y,clu)
            if len(clu) > 0:
                cls.append(clu)
    return(cls)

In [14]:
def boundary_pos():
    """ Creates dictionary, which labels the spins of lattice (assigning a number from 0 to N^2) and registers their position
        in the lattice
        
        Returns:
            pos: dtype dictionary
                dictionary of (boundary) spins which registers their position in the lattice     
    """
    tel = 0
    pos = {i: [] for i in range(N**2)}
    for j in range(N):
        for i in range(N):
            pos[tel] = [i,j,0]
            tel += 1
    return(pos)

In [15]:
def construct_graph(lat):
    """ Construct graph from spin lattice according to the AoSD procedure
        
        Input: 
            lat: dtype numpy array
                [N,N] array containing only 1 and -1 values representing the spin configuration
            
        Returns: 
            Gr: dtype networkx graph
                AoSD graph 
            
            pos: dtype dictionary
                extended boundary position dictionary to a dictionary of all the nodes/spins of the network, i.e. all 
                nodes/spins of the network are labeled by a number and the dictionary registers their (3D) position in space
                
            alm: dtype scalar
                allometric scale of the constructed network
    """
    # Initialize
    lats = []
    lats.append(lat)
    shell = 0
    n_nodes_shell = np.array(lats[shell]).size
    add = 0
    dz = 1
    alm = {}

    Gr = nx.Graph()
    for i in range(N**2):
        Gr.add_node(i)

    pos = boundary_pos()
            
    while(n_nodes_shell>1):
        new_lat = []
        if shell == 0:
            # perform step 1 of the coarse grain procedure with 2D boundary lattice
            clusters = cluster_decomp(lats[shell])
            # go through clusters
            for i in range(len(clusters)):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                new_pos = list(np.round(np.mean(clusters[i],axis=0),1))
                new_pos.append((shell+1)*dz)
                pos.update({n_Gr: new_pos})
                a = len(clusters[i])+1; c = a+len(clusters[i])
                alm.update({n_Gr: [a,c]})
                for j in clusters[i]:
                    value += lats[shell][j[0],j[1]]
                    lat_node = j[0]+j[1]*N
                    Gr.add_edge(n_Gr,lat_node)
                new_lat.append(value)
        elif shell%2 == 0:
            # perform step 1 of the coarse grain procedure (create domain branches)
            clusters = oneD_cluster_decomp(lats[shell])
            # go through clusters
            for i in range(len(clusters)):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                new_pos = []
                a = 1; c = 0
            
                for j in clusters[i]:
                    value += lats[shell][j]
                    Gr.add_edge(n_Gr,j+add)
                    new_pos.append(pos[j+add])
                    a += alm[j+add][0]; c += alm[j+add][1]
                new_pos = list(np.round(np.mean(np.array(new_pos),axis=0),1))
                new_pos[2] = (shell+1)*dz
                pos.update({n_Gr: new_pos})
                c += a
                alm.update({n_Gr: [a,c]})
                new_lat.append(value)
        else:
            # peform step 2 of the coarse grain procedure (average over domains)
            r = n_nodes_shell%2
            l = n_nodes_shell-r
            for k in np.arange(0,l,2):
                value = 0
                n_Gr = nx.number_of_nodes(Gr); Gr.add_node(n_Gr)
                #print('k+add',k+add)
                #print('alm', alm)
                new_pos = []
                new_pos.append(pos[k+add]); new_pos.append(pos[k+1+add])
                a = alm[k+add][0]+alm[k+1+add][0]+1; c = alm[k+add][1]+alm[k+1+add][1] 
                value = lats[shell][k]+lats[shell][k+1]
                
                # if three clusters remain:
                if (k+1)==(l-1) and r!=0:
                    value += lats[shell][l]
                    Gr.add_edge(n_Gr,l+add)
                    new_pos.append(pos[l+add])
                    a += alm[l+add][0]; c += alm[l+add][1]
                
                if value == 0:
                    value = np.random.choice([-1,1])
                else:
                    value = value/abs(value)
                Gr.add_edge(n_Gr,k+add); Gr.add_edge(n_Gr,k+1+add)
                new_pos = list(np.round(np.mean(np.array(new_pos),axis=0),1))
                new_pos[2] = (shell+1)*dz
                pos.update({n_Gr: new_pos})
                c += a
                alm.update({n_Gr: [a,c]})
                new_lat.append(value)
    
        lats.append(new_lat)
        shell += 1
        add += n_nodes_shell; n_nodes_shell = np.array(lats[shell]).size
    alm = alm[n_Gr]
    return(Gr, pos, alm)

In [16]:
def bulk_plot(ax, Gr, pos):
    """ Creates 3D plot of the network (boundary+bulk), i.e. its nodes and edges
    
        Input:
            ax: dtype matplotlib axis (projection 3d)
            
            Gr: dtype networkx graph
                the AoSD graph
            
            pos: dtype dictionary
                position dictionary of the nodes/spins of the network    
    """
    hs = 1
    ax.cla()
    # Loop on the pos dictionary to extract the x,y,z coordinates of each node
    for key, value in pos.items():
        xi = value[0]; yi = value[1]; zi = value[2]
        ax.scatter(hs*xi, hs*yi, zi, s=10, c='r')
        ax.text(hs*xi,hs*yi,zi,key)
        ax.set_zlim3d(0,4)
    
    # Loop on the list of edges to get the x,y,z, coordinates of the connected nodes
    # Those two points are the extrema of the line to be plotted
    for i,j in enumerate(Gr.edges()):
        x = np.array((pos[j[0]][0], pos[j[1]][0]))
        y = np.array((pos[j[0]][1], pos[j[1]][1]))
        z = np.array((pos[j[0]][2], pos[j[1]][2]))
    
        # Plot the connecting lines
        ax.plot(hs*x, hs*y, z, c='k',linewidth=0.8,alpha=0.6)
        
    ax.set_axis_off()
    return()

# Exhibition

In [17]:
def exhibition(temp, save):
    t_sim = 20
    global T
    T = temp
    #global exp
    #exp = np.array([np.exp(-8/T),np.exp(-4/T),1,np.exp(4/T),np.exp(8/T)]) # Store possible values of the acceptance probability (only
    # need first two values for metropolis)
    global A_wolff
    A_wolff = np.exp(-2/T) # Needed for acceptance value for Wolff
    
    # Initialize lattice
    lattice = -1*np.ones((N,N))  
    #lattice = np.random.choice([1,-1],size=(N,N))

    # Calculate energy and magnetisation by performing a sweep over the whole lattice (only once, as at each step it is updated)
    E = energy_calc(lattice); M = np.sum(lattice) 

    # Initialise figure
    %matplotlib notebook
    fig = plt.figure(figsize=(10,5)); plt.title('N=%d, T=%.2f'%(N**2,temp)); plt.axis('off') 
    grid = plt.GridSpec(10, 10, hspace=0.2, wspace=0.2)
    ax1 = fig.add_subplot(grid[2:8,:3]); ax1.set_axis_off()
    ax2 = fig.add_subplot(grid[:,3:],projection='3d'); ax2.set_zlim3d(0,4)

    # Set boundary plot
    x, y = np.meshgrid(range(N),range(N), indexing='ij')
    # Add labels
    pos = boundary_pos()
    for i in pos:
        ax1.annotate(i,(pos[i][0],pos[i][1]))

    for j in range(t_sim):
        # Perform one MCS using metropolis algorithm:
        #for k in range(N**2):
            #lattice, E, M = metropolis(lattice,E,M) 
            
        # Perform cluster flip using wolff algorithm:
        lattice, E, M = Wolff(lattice,E,M) 
        
        # Update boundary
        ax1.scatter(x,y,s=100,c=lattice)
    
        # Update bulk
        bulkG, positions, allometric = construct_graph(lattice)
        bulk_plot(ax2, bulkG, positions)
    
        fig.canvas.draw()
        #time.sleep(0.1)
        
    if save:
        plt.savefig('2DcorrespondenceN=%dT=%d'%(N**2,temp))

In [18]:
N = 6
wg.interact_manual(exhibition, temp=(0.1,6,0.01), save=False)

interactive(children=(FloatSlider(value=3.0500000000000003, description='temp', max=6.0, min=0.1, step=0.01), …

<function __main__.exhibition(temp, save)>